In [1]:
import pandas as pd
import numpy as np

In [2]:
def preprocessing_data(person_number, calories, distance, l_act, m_act, r_heart, sedentary, sleep_score, sleep, steps, hr_zone, v_act, wellness):
    #data frequency to date using sum
    calories_processed = calories.resample('D', on = 'dateTime').value.sum() 
    distance_processed = distance.resample('D', on = 'dateTime').value.sum()
    steps_processed = steps.resample('D', on = 'dateTime').value.sum()
    print('Sum Calories, Distance, Steps values per day')
    
    #resting heart rate to simple df
    r_heart['value']=r_heart['value'].apply(lambda x: x['value'])
    r_heart_processed=r_heart
    print('Process Resting Heart Rate to data frame')
    
    #delete time in sleep score time stamp
    sleep_score['timestamp']=sleep_score['timestamp'].apply(lambda x: x[:10])
    #drop unnecessary column
    sleep_score_processed = sleep_score.drop(columns='sleep_log_entry_id')
    print('Delete time in Sleep Score Timestamp and drop unnecessary columns in Sleep Score table')
    
    #dictionary with heart rate zone minutes to df
    hr_zone_value = pd.DataFrame([eval(str(a).replace('{\'valuesInZones\': ','').replace('}}','}')) for a in hr_zone['value']])
    hr_zone_processed = pd.concat([hr_zone['dateTime'], hr_zone_value], axis=1)
    print('Process Heart Rate Zone to data frame')
    
    #unify index
    l_act_processed = l_act.set_index('dateTime')
    m_act_processed = m_act.set_index('dateTime')
    r_heart_processed = r_heart_processed.set_index('dateTime')
    sedentary_processed = sedentary.set_index('dateTime')
    sleep_score_processed = sleep_score_processed.set_index('timestamp')
    sleep_processed = sleep.set_index('dateOfSleep')
    hr_zone_processed = hr_zone_processed.set_index('dateTime')
    v_act_processed = v_act.set_index('dateTime')
    #index to DateTimeIndex
    sleep_score_processed.index = pd.to_datetime(sleep_score_processed.index)
    print('Set all table data index with date')
   
    if person_number in ['08','14']:
        hr_zone_processed = hr_zone_processed.drop(['IN_CUSTOM_ZONE','BELOW_CUSTOM_ZONE','ABOVE_CUSTOM_ZONE'],axis=1)
        print('Drop extra columns in Heart Rate Zone table')
        
    #drop unnecessary columns
    sleep_processed = sleep_processed.drop(['logId','startTime','endTime','type','infoCode','levels', 'mainSleep'], axis=1)
    #index to DateTimeIndex
    sleep_processed.index = pd.to_datetime(sleep_processed.index)
    print('Drop unnecessary columns in Sleep table')
    
    #read prediction target
    wellness['effective_time_frame'] = wellness['effective_time_frame'].apply(lambda x: x[:10])
    wellness = wellness.rename(columns={"effective_time_frame": "dateTime"})
    readiness = wellness.loc[:,['dateTime','readiness']].set_index('dateTime')
    readiness.index = pd.to_datetime(readiness.index)
    print('Create Readiness column as prediction target from Wellness table')
    
    #drop duplicated records
    readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed = [x[~x.index.duplicated(keep='first')] for x in [readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed]]
    print('Drop duplicate records')
    
    #concat data
    P = pd.concat([readiness, calories_processed,distance_processed,l_act_processed,m_act_processed,r_heart_processed,sedentary_processed,sleep_score_processed,sleep_processed,steps_processed,hr_zone_processed,v_act_processed], axis=1)
    print('Merge all table data for participant ' + person_number + ' into one big table')
    
    #rename columns
    P.columns=['readiness','calories','distance','l_act','m_act','resting_heart','sedentary','sleep_overall_score','sleep_composition_score',
             'sleep_revitalization_score', 'sleep_duration_score',
             'deep_sleep_in_minutes', 'sleep_resting_heart_rate','sleep_restlessness',
             'sleep_duration','sleep_minutesToFallAsleep','sleep_minutesAsleep','sleep_minutesAwake',
             'sleep_minutesAfterWakeup','sleep_timeInBed','sleep_efficiency',
             'steps',
             'hr_below_zone1', 'hr_zone1', 'hr_zone3','hr_zone2','v_act']
    
    return P